In [65]:
import numpy as np

In [202]:
class SimplexSolver(object):
    def __init__(self,c,A,b):
        m,n = A.shape
        x = np.zeros(n)
        # Check if feasible
        feasible = True
        for i in range(m):
            k = b[i]-A[i,:]@x
            if k<0:
                feasible = False
                break
        if feasible==False:
            raise ValueError("Infeasible problem")
        # Keep track of variables
        var = np.empty(n+m)
        var[:n] = x
        var[n:] = b
        # index list for basic and nonbasic
        L = np.empty_like(var)
        L[:m] = np.arange(n,n+m,1)
        L[m:] = np.arange(0,n,1)
        
        self.A = A
        self.c = c
        self.b = b
        self.var = var
        self.L = L
        
    def init_tableau(self):
        A = self.A
        b = self.b
        c = self.c
        m,n = A.shape
        Abar = np.hstack((A,np.eye(m)))
        cbarT = np.hstack((c,np.zeros(m)))
        col1 = np.hstack((np.array([0]),b))
        col2 = np.vstack((-cbarT,Abar))
        col3 = np.hstack((np.array([1]), np.zeros(m)))
        T = np.vstack((col1,col2.T,col3)).T
        self.T = T
    
    def pivot(self):
        T = np.copy(self.T)
        m,n = T.shape
        L = np.copy(self.L)
        # Find pivot column and rows
        col = np.argmin(T[0,1:])+1
        check = np.argmax(T[1:,col])+1
        if T[check,col]<=0:
            raise ValueError('Solution is unbounded')
        no_zeros = np.copy(T[1:,col])
        no_zeros[no_zeros==0]=-1
        ratios = T[1:,0]/no_zeros
        ratios[T[1:,col]<=0] = np.inf
        row = np.argmin(ratios)+1
        ent = np.where(L==(col-1))[0]
        # Swap entering and leaving
        L[ent], L[row-1] = L[row-1], L[ent]
        # Operations on tableau
        pivot = T[row,col]
        T[row,:] = T[row,:]/pivot
        for i in range(m):
            if i!=row:
                k = T[i,col]
                T[i,:] = T[i,:]-k*T[row,:]
        # Update L and T
        return T,L
    
    def solve(self):
        self.init_tableau()
        check = np.argmin(self.T[0,1:])+1
        sol = self.T[0,check]
        # Check if solution found
        while sol < 0: 
            T1, L1 = self.pivot()
            self.T = T1
            self.L = L1
            check = np.argmin(self.T[0,1:])+1
            sol = self.T[0,check]
        T = self.T
        L = self.L.astype(int)
        T = np.round(T,2).astype(str)
        optimum = T[0,0]
        m,n = self.A.shape
        basic = dict(zip(L[:m],T[1:,0]))
        nonbasic = dict(zip(L[m:],np.zeros(n).astype(str)))
        return (optimum, basic, nonbasic)

In [203]:
c = np.array([3., 2])
b = np.array([2., 5, 7])
A = np.array([[1., -1], [3, 1], [4, 3]])
solver = SimplexSolver(c,A,b)
sol = solver.solve()
print(sol)

('5.2', {2: '0.6', 0: '1.6', 1: '0.2'}, {3: '0.0', 4: '0.0'})


# Product Mix Problem

In [204]:
data = np.load('/Users/rubyzhang/Desktop/UChicago/OSML/BootCamp2017/Computation/Wk4_DifIntOpt/productMix.npz')
A = data['A']
c = data['p']
b = np.hstack((data['m'],data['d']))
i,j = A.shape
for k in range(j):
    A = np.vstack((A,np.eye(1,j,k)))
solver = SimplexSolver(c,A,b)
sol = solver.solve()
print(sol)

('7453.6', {1: '6.19', 3: '1.79', 6: '0.97', 0: '10.0', 8: '13.81', 2: '12.0', 10: '8.21'}, {7: '0.0', 4: '0.0', 9: '0.0', 5: '0.0'})
